In [1]:
# !pip install nba_api
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import LeagueGameFinder
from nba_api.stats.endpoints import BoxScoreTraditionalV3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# Identify all OT games

In [16]:
# ...existing code...
# 1. Get all games from 2018-19 and filter for Overtime
def get_ot_games_2018_19():
    print("Fetching 2018-19 Regular Season games...")
    game_finder = LeagueGameFinder(season_nullable='2018-19', season_type_nullable='Regular Season')
    games = game_finder.get_data_frames()[0]
    
    # Helper to parse minutes
    def parse_minutes(min_val):
        if pd.isna(min_val): return 0
        min_str = str(min_val)
        if ':' in min_str:
            return int(min_str.split(':')[0])
        return int(float(min_str))

    games['MIN_CLEAN'] = games['MIN'].apply(parse_minutes)
    
    # Filter for games longer than 250 minutes (Standard is 240, OT is usually 265)
    ot_games_rows = games[games['MIN_CLEAN'] >= 250].copy()
    
    # Get unique game IDs
    unique_ot_ids = ot_games_rows['GAME_ID'].unique().tolist()
    
    print(f"Found {len(unique_ot_ids)} unique overtime games.")
    return unique_ot_ids

# ...existing code...
# ...existing code...
# 2. Get starters for OT
def get_ot_starting_lineup(game_id):
    try:
        # Get players in Period 5 (OT1)
        box = BoxScoreTraditionalV3(game_id=game_id, start_period=5, end_period=5, range_type=0, timeout=30)
        
        # MANUAL PARSING: Avoid get_data_frames()
        data = box.get_dict()
        
        if 'boxScoreTraditional' not in data:
            return pd.DataFrame()
        
        bs = data['boxScoreTraditional']
        players_list = []
        
        # V3 Structure: boxScoreTraditional -> homeTeam/awayTeam -> players
        if 'homeTeam' in bs and 'players' in bs['homeTeam']:
            for p in bs['homeTeam']['players']:
                # Flatten the nested structure
                flat_player = {
                    'personId': p.get('personId'),
                    'firstName': p.get('firstName'),
                    'familyName': p.get('familyName'),
                    'teamId': bs['homeTeamId'],
                    'teamTricode': bs['homeTeam'].get('teamTricode')
                }
                if 'statistics' in p:
                    flat_player.update(p['statistics'])
                players_list.append(flat_player)
        
        if 'awayTeam' in bs and 'players' in bs['awayTeam']:
            for p in bs['awayTeam']['players']:
                flat_player = {
                    'personId': p.get('personId'),
                    'firstName': p.get('firstName'),
                    'familyName': p.get('familyName'),
                    'teamId': bs['awayTeamId'],
                    'teamTricode': bs['awayTeam'].get('teamTricode')
                }
                if 'statistics' in p:
                    flat_player.update(p['statistics'])
                players_list.append(flat_player)
            
        if not players_list:
            return pd.DataFrame()
            
        players_df = pd.DataFrame(players_list)
        
        # 1. Normalize column names to uppercase
        players_df.columns = [c.upper() for c in players_df.columns]
        
        # 2. Map V3 columns to standard names
        rename_map = {
            'PERSONID': 'PLAYER_ID',
            'TEAMID': 'TEAM_ID',
            'TEAMTRICODE': 'TEAM_ABBREVIATION',
            'MINUTES': 'MIN'
        }
        players_df = players_df.rename(columns=rename_map)
        
        # 3. Handle Player Name
        if 'PLAYER_NAME' not in players_df.columns:
            if 'FIRSTNAME' in players_df.columns and 'FAMILYNAME' in players_df.columns:
                players_df['PLAYER_NAME'] = players_df['FIRSTNAME'] + ' ' + players_df['FAMILYNAME']
        
        # 4. Add GAME_ID
        players_df['GAME_ID'] = game_id
        
        # 5. Filter for active players (those with minutes > 0)
        if 'MIN' not in players_df.columns:
            return pd.DataFrame()
            
        active_ot_players = players_df[players_df['MIN'].notna()].copy()
        
        # Clean minutes - remove "0:00" entries
        if active_ot_players['MIN'].dtype == object:
             active_ot_players = active_ot_players[active_ot_players['MIN'].str.contains(r'[1-9]', regex=True)]

        if active_ot_players.empty: 
            return pd.DataFrame()

        candidate_ids = set(active_ot_players['PLAYER_ID'].tolist())
        
        # 6. Try to check substitutions (may fail for old games)
        try:
            pbp = PlayByPlayV2(game_id=game_id, start_period=5, end_period=5, timeout=30)
            pbp_df = pbp.play_by_play.get_data_frame()
            
            subs = pbp_df[pbp_df['EVENTMSGTYPE'] == 8]
            non_starters = set()
            
            for _, row in subs.iterrows():
                if row['PCTIMESTRING'] == '05:00':
                    non_starters.add(row['PLAYER1_ID']) 
                else:
                    non_starters.add(row['PLAYER2_ID']) 
            
            starter_ids = candidate_ids - non_starters
            starters_df = active_ot_players[active_ot_players['PLAYER_ID'].isin(starter_ids)].copy()
        except:
            # Fallback: Just return top 5 players by minutes for each team
            starters_df = active_ot_players.sort_values('MIN', ascending=False).groupby('TEAM_ID').head(5)
        
        # Ensure we return standard columns
        cols_to_return = ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME']
        available_cols = [c for c in cols_to_return if c in starters_df.columns]
        
        return starters_df[available_cols]

    except Exception as e:
        print(f"Error processing game {game_id}: {e}")
        return pd.DataFrame()
# ...existing code...
# --- Execute and Save ---
ot_game_ids = get_ot_games_2018_19()
all_starters = []

print(f"Extracting lineups for {len(ot_game_ids)} games...")
for i, game_id in enumerate(ot_game_ids):
    print(f"Processing {i+1}/{len(ot_game_ids)}: {game_id}")
    df = get_ot_starting_lineup(game_id)
    if not df.empty:
        all_starters.append(df)
    time.sleep(1.0) # Increased delay to avoid API errors

if all_starters:
    final_df = pd.concat(all_starters, ignore_index=True)
    final_df.to_csv('ot_starting_lineups_2018_19.csv', index=False)
    print("Success! Saved 'ot_starting_lineups_2018_19.csv'")
else:
    print("No data found.")

Fetching 2018-19 Regular Season games...
Found 67 unique overtime games.
Extracting lineups for 67 games...
Processing 1/67: 0021801229
Processing 2/67: 0021801195
Processing 3/67: 0021801201
Processing 4/67: 0021801189
Processing 5/67: 0021801145
Processing 6/67: 0021801132
Processing 7/67: 0021801107
Processing 8/67: 0021801106
Processing 9/67: 0021801080
Processing 10/67: 0021801070
Processing 11/67: 0021801072
Processing 12/67: 0021801069
Processing 13/67: 0021801056
Processing 14/67: 0021801036
Processing 15/67: 0021800988
Processing 16/67: 0021800976
Processing 17/67: 0021800949
Processing 18/67: 0021800928
Processing 19/67: 0021800912
Processing 20/67: 0021800920
Processing 21/67: 0021800881
Processing 22/67: 0021800853
Processing 23/67: 0021800769
Processing 24/67: 0021800760
Processing 25/67: 0021800686
Processing 26/67: 0021800670
Processing 27/67: 0021800657
Processing 28/67: 0021800659
Processing 29/67: 0021800639
Processing 30/67: 0021800619
Processing 31/67: 0021800569
Pr

# Map Ratings

In [18]:
# Check if we have the data in memory
if 'final_df' in locals() or 'final_df' in globals():
    print("final_df exists in memory!")
    print(f"Shape: {final_df.shape}")
    print(final_df.head(10))
    
    # Save to OneDrive Documents
    import os
    documents_path = r"C:\Users\ariel\OneDrive\Documents"
    csv_path = os.path.join(documents_path, 'ot_starting_lineups_2018_19.csv')
    final_df.to_csv(csv_path, index=False)
    print(f"\nFile saved to: {csv_path}")
    print(f"Total rows: {len(final_df)}")
    print(f"Total unique games: {final_df['GAME_ID'].nunique()}")
else:
    print("final_df does not exist. The extraction may have failed.")

final_df exists in memory!
Shape: (670, 5)
      GAME_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_ID          PLAYER_NAME
0  0021801229  1610612746               LAC    1627820       Tyrone Wallace
1  0021801229  1610612746               LAC    1628414  Sindarius Thornwell
2  0021801229  1610612762               UTA    1628960        Grayson Allen
3  0021801229  1610612762               UTA    1627777        Georges Niang
4  0021801229  1610612762               UTA    1628513      Naz Mitrou-Long
5  0021801229  1610612746               LAC    1627826          Ivica Zubac
6  0021801229  1610612746               LAC    1629013        Landry Shamet
7  0021801229  1610612762               UTA     202327            Ekpe Udoh
8  0021801229  1610612762               UTA     200757      Thabo Sefolosha
9  0021801229  1610612746               LAC     203210       JaMychal Green

File saved to: C:\Users\ariel\OneDrive\Documents\ot_starting_lineups_2018_19.csv
Total rows: 670
Total unique games: 67


In [23]:
import unicodedata

# Function to remove accents from text
def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalize to NFD (decomposed form) and filter out combining characters
    nfd = unicodedata.normalize('NFD', text)
    return ''.join(char for char in nfd if unicodedata.category(char) != 'Mn')

# Clean player names in final_df
final_df['PLAYER_NAME_CLEAN'] = final_df['PLAYER_NAME'].apply(remove_accents)

print("Cleaned player names:")
print(final_df[['PLAYER_NAME', 'PLAYER_NAME_CLEAN']].drop_duplicates().tail(20))




Cleaned player names:
                PLAYER_NAME       PLAYER_NAME_CLEAN
559          T.J. McConnell          T.J. McConnell
579  Michael Kidd-Gilchrist  Michael Kidd-Gilchrist
580             Aron Baynes             Aron Baynes
581            Semi Ojeleye            Semi Ojeleye
583      Guerschon Yabusele      Guerschon Yabusele
589            Jaylen Brown            Jaylen Brown
590            Lance Thomas            Lance Thomas
595           Allonzo Trier           Allonzo Trier
597            Enes Freedom            Enes Freedom
598           Cameron Payne           Cameron Payne
605         Josh Richardson         Josh Richardson
609         Rodney McGruder         Rodney McGruder
613             Gary Harris             Gary Harris
619      Chandler Hutchison      Chandler Hutchison
622       Langston Galloway       Langston Galloway
625            Caris LeVert            Caris LeVert
630           Davis Bertans           Davis Bertans
631        Quincy Pondexter        Quincy 

In [24]:
# Load RAPM data
rapm_path = r"C:\Users\ariel\OneDrive\Documents\rapm.csv"
rapm_df = pd.read_csv(rapm_path)

print("\nRAPM data loaded!")
print(f"Shape: {rapm_df.shape}")
print("\nFirst few rows:")
print(rapm_df.head())
print("\nColumn names:")
print(rapm_df.columns.tolist())

# Clean RAPM player names too
rapm_df['playerName_clean'] = rapm_df['playerName'].apply(remove_accents)

# Merge the datasets
merged_df = final_df.merge(
    rapm_df,
    left_on='PLAYER_NAME_CLEAN',
    right_on='playerName_clean',
    how='left'
)

print(f"\nMerged data shape: {merged_df.shape}")
print(f"Players with RAPM data: {merged_df['playerName'].notna().sum()}/{len(merged_df)}")
print("\nSample merged data:")


RAPM data loaded!
Shape: (529, 10)

First few rows:
   Unnamed: 0  playerId       playerName  RAPM  RAPM_Rank  RAPM__Def  \
0           0    101106     Andrew Bogut -0.10      280.0      -0.15   
1           1    101107  Marvin Williams -0.07      269.0      -0.37   
2           2    101108       Chris Paul  1.69       42.0       1.13   
3           3    101109   Raymond Felton  0.03      234.0       0.74   
4           4    101112    Channing Frye -1.78      503.0      -1.21   

   RAPM__Def_Rank  RAPM__Off  RAPM__Off_Rank  RAPM__intercept  
0           324.0       0.05           228.0           111.32  
1           404.0       0.31           145.0           111.32  
2            25.0       0.56            97.0           111.32  
3            72.0      -0.71           455.0           111.32  
4           514.0      -0.57           431.0           111.32  

Column names:
['Unnamed: 0', 'playerId', 'playerName', 'RAPM', 'RAPM_Rank', 'RAPM__Def', 'RAPM__Def_Rank', 'RAPM__Off', 'RAPM__Of

In [27]:

# Load RAPM data
rapm_path = r"C:\Users\ariel\OneDrive\Documents\rapm.csv"
rapm_df = pd.read_csv(rapm_path)

print("\nRAPM data loaded!")
print(f"Shape: {rapm_df.shape}")
print("\nFirst few rows:")
print(rapm_df.head())
print("\nColumn names:")
print(rapm_df.columns.tolist())

# Clean RAPM player names too
rapm_df['playerName_clean'] = rapm_df['playerName'].apply(remove_accents)

# Merge the datasets
merged_df = final_df.merge(
    rapm_df,
    left_on='PLAYER_NAME_CLEAN',
    right_on='playerName_clean',
    how='left'
)

print(f"\nMerged data shape: {merged_df.shape}")
print(f"Players with RAPM data: {merged_df['playerName'].notna().sum()}/{len(merged_df)}")
print("\nSample merged data:")
print(merged_df[['PLAYER_NAME', 'TEAM_ABBREVIATION', 'playerName', 'RAPM__Def', 'RAPM__Off']].head(10))


RAPM data loaded!
Shape: (529, 10)

First few rows:
   Unnamed: 0  playerId       playerName  RAPM  RAPM_Rank  RAPM__Def  \
0           0    101106     Andrew Bogut -0.10      280.0      -0.15   
1           1    101107  Marvin Williams -0.07      269.0      -0.37   
2           2    101108       Chris Paul  1.69       42.0       1.13   
3           3    101109   Raymond Felton  0.03      234.0       0.74   
4           4    101112    Channing Frye -1.78      503.0      -1.21   

   RAPM__Def_Rank  RAPM__Off  RAPM__Off_Rank  RAPM__intercept  
0           324.0       0.05           228.0           111.32  
1           404.0       0.31           145.0           111.32  
2            25.0       0.56            97.0           111.32  
3            72.0      -0.71           455.0           111.32  
4           514.0      -0.57           431.0           111.32  

Column names:
['Unnamed: 0', 'playerId', 'playerName', 'RAPM', 'RAPM_Rank', 'RAPM__Def', 'RAPM__Def_Rank', 'RAPM__Off', 'RAPM__Of

In [29]:
# Get game results using BoxScoreTraditionalV3
def get_game_result_v3(game_id):
    """Get final score and winner for a game using V3 API"""
    try:
        box = BoxScoreTraditionalV3(game_id=game_id, timeout=30)
        data = box.get_dict()
        
        if 'boxScoreTraditional' not in data:
            return None
        
        bs = data['boxScoreTraditional']
        
        result = {
            'GAME_ID': game_id,
            'HOME_TEAM_ID': bs.get('homeTeamId'),
            'HOME_TEAM_ABB': bs.get('homeTeam', {}).get('teamTricode'),
            'HOME_SCORE': bs.get('homeTeam', {}).get('statistics', {}).get('points'),
            'AWAY_TEAM_ID': bs.get('awayTeamId'),
            'AWAY_TEAM_ABB': bs.get('awayTeam', {}).get('teamTricode'),
            'AWAY_SCORE': bs.get('awayTeam', {}).get('statistics', {}).get('points'),
        }
        
        # Determine winner
        if result['HOME_SCORE'] and result['AWAY_SCORE']:
            result['HOME_WIN'] = 1 if result['HOME_SCORE'] > result['AWAY_SCORE'] else 0
        else:
            result['HOME_WIN'] = None
            
        return result
    except Exception as e:
        print(f"Error getting result for {game_id}: {e}")
        return None

# Get all game results
print("Fetching game results...")
game_results = []
for i, game_id in enumerate(merged_df['GAME_ID'].unique()):
    print(f"Processing game {i+1}/{merged_df['GAME_ID'].nunique()}: {game_id}")
    result = get_game_result_v3(game_id)
    if result:
        game_results.append(result)
    time.sleep(0.6)

game_results_df = pd.DataFrame(game_results)
print(f"\nGame results collected: {len(game_results_df)} games")
print(game_results_df.head())

Fetching game results...
Processing game 1/67: 0021801229
Processing game 2/67: 0021801195
Processing game 3/67: 0021801201
Processing game 4/67: 0021801189
Processing game 5/67: 0021801145
Processing game 6/67: 0021801132
Processing game 7/67: 0021801107
Processing game 8/67: 0021801106
Processing game 9/67: 0021801080
Processing game 10/67: 0021801070
Processing game 11/67: 0021801072
Processing game 12/67: 0021801069
Processing game 13/67: 0021801056
Processing game 14/67: 0021801036
Processing game 15/67: 0021800988
Processing game 16/67: 0021800976
Processing game 17/67: 0021800949
Processing game 18/67: 0021800928
Processing game 19/67: 0021800912
Processing game 20/67: 0021800920
Processing game 21/67: 0021800881
Processing game 22/67: 0021800853
Processing game 23/67: 0021800769
Processing game 24/67: 0021800760
Processing game 25/67: 0021800686
Processing game 26/67: 0021800670
Processing game 27/67: 0021800657
Processing game 28/67: 0021800659
Processing game 29/67: 002180063